<a href="https://colab.research.google.com/github/subramanyatn/Vehicle-Number-plate-detection/blob/main/Final_MiniProject_Image.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install opencv-python
!pip install opencv-contrib-python
!pip install matplotlib
!pip install Pillow
!pip install pytesseract
!sudo apt install tesseract-ocr


  Created wheel for pytesseract: filename=pytesseract-0.3.8-py2.py3-none-any.whl size=14071 sha256=aa219e426113c03e8382cc730e213ccc2c19bccd8e421a5c29c02e69ceb4aa39
  Stored in directory: /root/.cache/pip/wheels/a2/66/45/88bf1b2d428817a006944b9730b27d6861b776e05a9e262bd4
Successfully built pytesseract
Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following additional packages will be installed:
  tesseract-ocr-eng tesseract-ocr-osd
The following NEW packages will be installed:
  tesseract-ocr tesseract-ocr-eng tesseract-ocr-osd
0 upgraded, 3 newly installed, 0 to remove and 39 not upgraded.
Need to get 4,795 kB of archives.
After this operation, 15.8 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu bionic/universe amd64 tesseract-ocr-eng all 4.00~git24-0e00fe6-1.2 [1,588 kB]
Get:2 http://archive.ubuntu.com/ubuntu bionic/universe amd64 tesseract-ocr-osd all 4.00~git24-0e00fe6-1.2 [2,989 kB]
Get:3 http:/

In [ ]:
import cv2
import random
import os
import pytesseract
import numpy as np
import glob
import matplotlib.pyplot as plt
%matplotlib inline
pytesseract.pytesseract.tesseract_cmd = (r'/usr/bin/tesseract')

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [ ]:
def plot_images(img1, img2, title1="", title2=""):
    fig = plt.figure(figsize=[15,15])
    ax1 = fig.add_subplot(121)
   
    ax1.imshow(img1, cmap="gray")
   

    ax1.set(xticks=[], yticks=[], title=title1)
   


    ax2 = fig.add_subplot(122)
    ax2.imshow(img2, cmap="gray")
    ax2.set(xticks=[], yticks=[], title=title2)

In [ ]:
ls -l '/content/gdrive/My Drive/Colab Notebooks/Images'

ls: cannot access '/content/gdrive/My Drive/Colab Notebooks/Images': No such file or directory


In [ ]:
images_dir = "/content/gdrive/My Drive/Colab Notebooks/Images"

In [ ]:
image_path = "{}/{}".format(images_dir, "HR26BU0380.png")

In [ ]:
image = cv2.imread(image_path)
    
rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
gray = cv2.cvtColor(rgb, cv2.COLOR_BGR2GRAY)

error: ignored

In [ ]:
plot_images(image, gray,title1="original image",title2="grayscale image")

In [ ]:
blur = cv2.bilateralFilter(gray, 11, 90, 90)



In [ ]:
plot_images(gray, blur, title1="grayscale image", title2="blur image")

In [ ]:
edges = cv2.Canny(blur, 30, 200)

In [ ]:
plot_images(blur, edges, title1="blur image", title2="edge_detection image")

In [ ]:
cnts, new = cv2.findContours(edges.copy(), cv2.RETR_LIST, cv2.CHAIN_APPROX_SIMPLE)


In [ ]:
cnts = sorted(cnts, key = cv2.contourArea, reverse = True)

NumberPlateCount = None

In [ ]:
image_copy = image.copy()

In [ ]:
_ = cv2.drawContours(image_copy, cnts, -1, (255,0,255), 2)


In [ ]:
plot_images(image, image_copy,title1=" image",title2=" image")

In [ ]:
cnts = sorted(cnts, key=cv2.contourArea, reverse=True)[:30]

In [ ]:
image_copy = image.copy()
_ = cv2.drawContours(image_copy, cnts, -1, (255,0,255), 2)

In [ ]:
plot_images(image, image_copy,title1=" image",title2=" image")

In [ ]:
plate = None
for c in cnts:
    perimeter = cv2.arcLength(c, True)
    

    edges_count = cv2.approxPolyDP(c, 0.02* perimeter, True)
    if len(edges_count) == 4:
        x,y,w,h = cv2.boundingRect(c)
        plate = image[y:y+h, x:x+w]
        break

cv2.imwrite("plate.png", plate)

In [ ]:
plot_images(plate, plate, title1=" image", title2=" image")

In [ ]:
gray = cv2.cvtColor(plate, cv2.COLOR_BGR2GRAY)
canny = cv2.Canny(plate, 100, 200)

plot_images(gray, canny, title1=" gray", title2=" canny")

In [ ]:
plateText = pytesseract.image_to_string(plate)
grayText = pytesseract.image_to_string(gray)

In [ ]:
print("plateText:",plateText)
print("grayText:",grayText)


In [ ]:
path_for_license_plates = "/content/gdrive/My Drive/Colab Notebooks/Images/*.*"
list_license_plates = [] 
predicted_license_plates = [] 
plateList = []

for path_to_license_plate in glob.glob(path_for_license_plates, recursive = True): 
  license_plate_file = path_to_license_plate.split("/")[-1] 
  license_plate, _ = os.path.splitext(license_plate_file) 
  
  
  list_license_plates.append(license_plate) 

  img = cv2.imread(path_to_license_plate)

  gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
  blur = cv2.bilateralFilter(gray, 11, 90, 90)
  edges = cv2.Canny(blur, 30, 200)
  cnts, new = cv2.findContours(edges.copy(), cv2.RETR_LIST, cv2.CHAIN_APPROX_SIMPLE)
  cnts = sorted(cnts, key = cv2.contourArea, reverse = True)
  NumberPlateCount = None
  image_copy = img.copy()
  _ = cv2.drawContours(image_copy, cnts, -1, (255,0,255), 2)
  cnts = sorted(cnts, key=cv2.contourArea, reverse=True)[:30]
  image_copy = img.copy()
  _ = cv2.drawContours(image_copy, cnts, -1, (255,0,255), 2)

  plate = None
  for c in cnts:
      perimeter = cv2.arcLength(c, True)
      edges_count = cv2.approxPolyDP(c, 0.02 * perimeter, True)
      if len(edges_count) == 4:
        x,y,w,h = cv2.boundingRect(c)
        plate = img[y:y+h, x:x+w]
        predicted_result = pytesseract.image_to_string(plate, lang ='eng', config='--psm 10 --oem 3 -c tessedit_char_whitelist=ABCDEFGHIJKLMNOPQRSTUVWXYZ0123456789')
        filter_predicted_result = "".join(predicted_result.split())
        predicted_license_plates.append(filter_predicted_result) 
        break

print(predicted_license_plates)        